In [1]:
import numpy as np
import time
import soundfile as sf

In [2]:
tube_length= 80
embochure = 20
fs = 44100
nbr_de_sec = 1
duration = fs*nbr_de_sec

In [3]:
def noise(duration):
    y = (np.random.rand(duration,1)-0.5)*2
    return y

def normalize(array):
    min = np.min(array)
    max = np.max(array)
    normalized_array = 2*(array-min)/(max-min) - 1
    return normalized_array

In [4]:
# Karjalainen 1991

# Noise ノイズ
burst_noise = noise(duration)

# Variables 変数
# Preallocate output array for efficiency
output = np.empty(duration)  
delayTube1 = np.empty(duration)
delayTube2 = np.empty(duration)
delayEmb = np.empty(duration)
feedbackGain = 0.77 #0.628
inputGain = 0.372

feedback = 0
delayff = 0
delayfr = 0
delayfb = 0
start = 0
end = 0

start = time.time()
for i in range(duration):
    # Input Noise　ノイズの入力
    instantSample = burst_noise[i] * inputGain

    # + feedback　フィードバック足し
    if i != 0:
        instantSample += feedback

    # Z-1　一サンプル遅延
    delayEmb[i] = instantSample
    if i > embochure:
        instantSample = delayEmb[i-embochure]

    # Non-linearity　シグモイド関数
    instantSample -= (instantSample**3)

    # + feedback　フィードバック足し
    if i != 0:
        instantSample += feedback

    # Variable delay-line　可変遅延線
    delayTube1[i] = instantSample
    if i > tube_length:
        instantSample = delayTube1[i-tube_length]
    
    # Ff　無限インパルス応答フィルター
    instantSample = instantSample * 0.5 + delayff * 0.5
    delayff = instantSample

    # Fr　逆の無限インパルス応答フィルター
    instantSample_to_fr = instantSample
    instantSample_to_fr1 = instantSample_to_fr * 0.5 + delayfr * (-0.5)
    delayfr = instantSample_to_fr1
    instantSample_to_fr_out = instantSample_to_fr - instantSample_to_fr1

    # Variable delay-line　可変遅延線
    delayTube2[i] = instantSample
    if i > tube_length:
        instantSample = delayTube2[i-tube_length]
        
    # Fb　無限インパルス応答フィルター
    instantSample_to_fb = instantSample * 0.5 + delayfb * 0.5
    delayfb = instantSample_to_fb

    # Feedback　フィードバック
    feedback = instantSample_to_fb * feedbackGain

    # Output　出力
    output[i] = instantSample_to_fr_out

normalized_output = normalize(output)
end = time.time()

In [5]:
# write output to file for listening
sf.write('output_Karjalainen91.wav', normalized_output, fs)

elapsed_time = end - start
print(elapsed_time)

0.8404684066772461
